In [1]:
from attomics import *
from json_tools import *
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
# Device
device= torch.device("cuda:1")

# Import data
df_mRNA = pd.read_parquet("data_attomics/mRNA.parquet")
df_label = pd.read_parquet("data_attomics/label.parquet")

# Pick multi-classes data
y_tumour = df_label.iloc[:, [0, 2]]

# Pick X_train
scaler=StandardScaler()
X_orignal = df_mRNA.iloc[:,1:df_mRNA.shape[1]]
X_scaled = scaler.fit_transform(X_orignal)

# Pick y_train
y_labels = set(y_tumour['cancer_type'])
y_dic = {label: index for index, label in enumerate(sorted(y_labels))}
y_numerical = np.array([y_dic[label] for label in y_tumour['cancer_type']])

X_train,X_testval,y_train,y_testval=train_test_split(X_scaled, y_numerical, 
                                                         test_size=0.3,random_state=2, stratify=y_numerical)

# Create weight loss function
unique, counts = np.unique(y_numerical, return_counts=True)
total_count = sum(counts)
weights = torch.tensor([total_count / c for c in counts], dtype=torch.float32).to(device)
loss_function_weight = nn.CrossEntropyLoss(weight=weights)

_,X_train_array,_,y_train_array=train_test_split(X_train, y_train, test_size=512, random_state=2, stratify=y_train)

X_train_tensor= torch.tensor(X_train_array)

unique,_ = np.unique(y_train, return_counts=True)
y_train_tensor = torch.tensor(y_train_array)

learning_rate = 0.0001

num_classes = len(unique)
feature_numbers = X_train_tensor.size(1)

# Move to GPU, initial optimizer
inputs= X_train_tensor.to(device=device)
labels = y_train_tensor.to(device=device)

In [3]:
n_group = 4000
init_group_size=math.ceil(feature_numbers/n_group)

In [6]:
encoder = AttOmicsEncoder(
    in_features=feature_numbers, n_group=n_group, 
    group_size_list=[math.ceil(init_group_size), math.ceil(init_group_size), math.ceil(init_group_size)],
    num_heads=1, flatten_output=True, pe=False, permutation=False, permu_inverse=False, 
    mem_efficient = True, K_linformer = 2,
).to(device)

encoder_output_d = math.ceil(init_group_size)

head = FullyConnectedNetwork(
    [FullyConnectedLayer(input_dim=n_group*encoder_output_d, 
                         output_dim=math.ceil((n_group* encoder_output_d)/16)),
     FullyConnectedLayer(input_dim=math.ceil((n_group* encoder_output_d)/16), 
                         output_dim=math.ceil((n_group* encoder_output_d)/64))]
).to(device)

model = AttOmics(encoder=encoder, head=head, num_classes=num_classes).to(device)

In [7]:
e = model(inputs)

torch.Size([512, 60000])


In [ ]:
modelname=f"Linformer_{n_group}_groups"
#model = encoder

def trace_handler(p):
    #p.export_memory_timeline(f"memory_recorder/{modelname}.html", device="cuda:1")
    p.export_memory_timeline(f"memory_recorder/{modelname}.json", device="cuda:1")

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA],
    profile_memory=True, 
    record_shapes=True,
    with_stack=True,
    schedule=torch.profiler.schedule(
        wait=1,
        warmup=3,
        active=3),
    on_trace_ready=trace_handler
) as p:
    for idx in range(7):
        p.step()
        e = model(inputs)

del model
torch.cuda.empty_cache()